In [1]:
# Library for SPARQL querying
!pip install SPARQLWrapper
# Pandas extension for geographical information
!pip install geopandas
# Map contexts for geographical plots
!pip install contextily

In [5]:
import pandas as pd
import json
from SPARQLWrapper import SPARQLWrapper, JSON

from shapely.geometry import Point
import geopandas
from geopandas import GeoDataFrame
from shapely import wkt

def get_sparql_dataframe(service, query, has_geo_data=False):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    Adapted from http://lawlesst.github.io/notebook/sparql-dataframe.html
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    data_dictionnary = result.convert()
    cols = data_dictionnary['head']['vars']

    out = []
    for row in data_dictionnary['results']['bindings']:
        item = []
        for c in cols:
            v = row.get(c, {}).get('value')
            t = row.get(c, {}).get('datatype')
            if has_geo_data and (t == 'http://www.opengis.net/ont/geosparql#wktLiteral'):
                v = wkt.loads(v)
            item.append(v)
        out.append(item)
    
    if has_geo_data:
        return GeoDataFrame(out, columns=cols, crs="EPSG:4326")  # setting the default coordinate reference system
    else:
        return pd.DataFrame(out, columns=cols)

## Access data

## Basic Exploration

## Take Away